## Disclaimer

This notebook is only to make the ILI levels datasets (granulated on weeks) with the energy and temperature datasets (granulated on hours). This is necessary for Tableau.

In [210]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [211]:
# load pennsylvania data
path = 'df_pennsylvania_flu.csv'
df_pennsylvania_flu = pd.read_csv(path)

# load illinois data
path = 'df_illinois_flu.csv'
df_illinois_flu = pd.read_csv(path)

display(df_pennsylvania_flu.head())
display()
display(df_illinois_flu.head())

,year,week,temp_mean,temp_min,temp_max,cold_hours,hot_hours,activity_level,activity_level_label,season
0,2012,40,15.683834,7.45,23.35,37,0,1,Minimal,2012-13
1,2012,41,7.817202,0.02,18.15,116,0,1,Minimal,2012-13
2,2012,42,12.615952,3.75,24.14,66,0,1,Minimal,2012-13
3,2012,43,15.317589,3.30,27.65,36,6,1,Minimal,2012-13
4,2012,44,4.587870,1.98,7.91,169,0,1,Minimal,2012-13


,year,week,temp_mean,temp_min,temp_max,cold_hours,hot_hours,activity_level,activity_level_label,season
0,2012,40,13.193105,1.77,25.47,46,1,1,Minimal,2012-13
1,2012,41,9.495625,1.37,18.76,99,0,1,Minimal,2012-13
2,2012,42,11.788571,4.77,20.76,77,0,1,Minimal,2012-13
3,2012,43,13.609077,2.20,25.85,61,4,1,Minimal,2012-13
4,2012,44,5.925178,1.51,11.45,164,0,2,Minimal,2012-13


Build a function to obtain the first and last date of a week in a given year:

In [212]:
def get_date_range(year, week):
    """a function to obtain the first and last date of a week in a given year"""
    # calculate the start date of the given week
    start_date = datetime.strptime(f'{year}-W{int(week)-1}-1', "%Y-W%W-%w").date()
    # calculate the end date of the given week
    # end_date = start_date + timedelta(days=6)
    end_date = '2017-12-01 00:00:00' # <--- this one to coincide with energy!!!!!
    return start_date, end_date

Build a dataframe with hourly frequency in the same range

In [213]:
# extract start date for pennsylvania
start_year = df_pennsylvania_flu['year'].min()
start_week = df_pennsylvania_flu[df_pennsylvania_flu['year'] == start_year]['week'].min()
start_date, _ = get_date_range(start_year, start_week)
start_date = pd.to_datetime(start_date)

# extract end date for pennsylvania
end_year = df_pennsylvania_flu['year'].max()
end_week = df_pennsylvania_flu[df_pennsylvania_flu['year'] == end_year]['week'].max()
_, end_date = get_date_range(end_year, end_week)
end_date = pd.to_datetime(end_date)
end_date = end_date + timedelta(hours=23)

# ----------- repeat for illinois data -----------
# extract start date for illinois
start_year = df_illinois_flu['year'].min()
start_week = df_illinois_flu[df_illinois_flu['year'] == start_year]['week'].min()
start_date, _ = get_date_range(start_year, start_week)
start_date = pd.to_datetime(start_date)

# extract end date for illinois
end_year = df_illinois_flu['year'].max()
end_week = df_illinois_flu[df_illinois_flu['year'] == end_year]['week'].max()
_, end_date = get_date_range(end_year, end_week)
end_date = pd.to_datetime(end_date)
end_date = end_date + timedelta(hours=23)

# check
print(f'Pennsylvania data starts from {start_date} and ends at {end_date}')
print()
print(f'Illinois data starts from {start_date} and ends at {end_date}')

Pennsylvania data starts from 2012-09-24 00:00:00 and ends at 2017-12-01 23:00:00

Illinois data starts from 2012-09-24 00:00:00 and ends at 2017-12-01 23:00:00


In [214]:
# create a new dataframe from start date to end date
df_pennsylvania_flu_hour = pd.date_range(start_date, end_date, freq='h')

# move the index to a column and reset index
df_pennsylvania_flu_hour = df_pennsylvania_flu_hour.to_frame(index=False)
df_pennsylvania_flu_hour.columns = ['date']

# ----------- repeat for illinois data -----------
# create a new dataframe from start date to end date
df_illinois_flu_hour = pd.date_range(start_date, end_date, freq='h')

# move the index to a column and reset index
df_illinois_flu_hour = df_illinois_flu_hour.to_frame(index=False)
df_illinois_flu_hour.columns = ['date']

# check
display(df_pennsylvania_flu_hour)
print()
display(df_illinois_flu_hour)

,date
0,2012-09-24 00:00:00
1,2012-09-24 01:00:00
2,2012-09-24 02:00:00
3,2012-09-24 03:00:00
4,2012-09-24 04:00:00
...,...
45475,2017-12-01 19:00:00
45476,2017-12-01 20:00:00
45477,2017-12-01 21:00:00
45478,2017-12-01 22:00:00


,date
0,2012-09-24 00:00:00
1,2012-09-24 01:00:00
2,2012-09-24 02:00:00
3,2012-09-24 03:00:00
4,2012-09-24 04:00:00
...,...
45475,2017-12-01 19:00:00
45476,2017-12-01 20:00:00
45477,2017-12-01 21:00:00
45478,2017-12-01 22:00:00


Convert the original 'year' column into datetime format

In [215]:
# convert year to datetime
df_pennsylvania_flu['date'] = pd.to_datetime(df_pennsylvania_flu['year'].astype(str), format='%Y')
df_pennsylvania_flu['date'] = df_pennsylvania_flu['date'] + pd.to_timedelta(df_pennsylvania_flu['week'] - 1, unit='W')
df_pennsylvania_flu['date'] = pd.to_datetime(df_pennsylvania_flu['date'], format='%Y-%m-%d %H:%M:%S')

df_illinois_flu['date'] = pd.to_datetime(df_illinois_flu['year'].astype(str), format='%Y')
df_illinois_flu['date'] = df_illinois_flu['date'] + pd.to_timedelta(df_illinois_flu['week'] - 1, unit='W')
df_illinois_flu['date'] = pd.to_datetime(df_illinois_flu['date'], format='%Y-%m-%d %H:%M:%S')

In [216]:
cols_to_add = df_pennsylvania_flu.columns

# merge the two dataframes
df_pennsylvania_flu_hour = pd.merge(df_pennsylvania_flu_hour, df_pennsylvania_flu[cols_to_add], how='left', on='date')
df_illinois_flu_hour = pd.merge(df_illinois_flu_hour, df_illinois_flu[cols_to_add], how='left', on='date')

In [217]:
df_pennsylvania_flu_hour.describe()

,date,year,week,temp_mean,temp_min,temp_max,cold_hours,hot_hours,activity_level
count,45481,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000
mean,2015-04-29 11:49:06.505134080,2014.829630,27.181481,10.885384,0.826411,21.077845,73.074074,11.785185,2.070370
min,2012-09-24 00:00:00,2012.000000,1.000000,-10.930587,-23.727333,-2.000000,0.000000,0.000000,1.000000
25%,2014-01-10 18:00:00,2014.000000,14.000000,3.287326,-5.785000,15.329833,2.000000,0.000000,1.000000
50%,2015-04-29 12:00:00,2015.000000,27.500000,12.028909,0.826833,23.012833,60.000000,0.000000,1.000000
75%,2016-08-15 06:00:00,2016.000000,41.000000,19.175382,9.423750,28.122500,143.500000,19.750000,2.000000
max,2017-12-01 23:00:00,2017.000000,53.000000,25.714697,19.550000,33.140000,192.000000,83.000000,10.000000
std,NaN,1.511004,15.176546,9.129928,9.955226,8.204174,66.351618,19.444281,2.182784


In [218]:
df_pennsylvania_flu.describe()

,year,week,temp_mean,temp_min,temp_max,cold_hours,hot_hours,activity_level,date
count,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270
mean,2014.829630,27.181481,10.885384,0.826411,21.077845,73.074074,11.785185,2.070370,2015-05-02 05:09:20
min,2012.000000,1.000000,-10.930587,-23.727333,-2.000000,0.000000,0.000000,1.000000,2012-09-30 00:00:00
25%,2014.000000,14.000000,3.287326,-5.785000,15.329833,2.000000,0.000000,1.000000,2014-01-16 18:00:00
50%,2015.000000,27.500000,12.028909,0.826833,23.012833,60.000000,0.000000,1.000000,2015-05-03 12:00:00
75%,2016.000000,41.000000,19.175382,9.423750,28.122500,143.500000,19.750000,2.000000,2016-08-17 06:00:00
max,2017.000000,53.000000,25.714697,19.550000,33.140000,192.000000,83.000000,10.000000,2017-11-26 00:00:00
std,1.511004,15.176546,9.129928,9.955226,8.204174,66.351618,19.444281,2.182784,NaN


In [219]:
# # remove only the first row of NaN values
# df_pennsylvania_flu_hour = df_pennsylvania_flu_hour[df_pennsylvania_flu_hour['year'].notna().cumsum() != 0]
# df_illinois_flu_hour = df_illinois_flu_hour[df_illinois_flu_hour['year'].notna().cumsum() != 0]

# fill the rest of the NaN values with the upper value
df_pennsylvania_flu_hour.ffill(inplace=True)
df_illinois_flu_hour.ffill(inplace=True)

# # change date to NY timezone
df_pennsylvania_flu_hour['date'] = df_pennsylvania_flu_hour['date'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')
df_illinois_flu_hour['date'] = df_illinois_flu_hour['date'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')

# check
display(df_pennsylvania_flu_hour.head())

,date,year,week,temp_mean,temp_min,temp_max,cold_hours,hot_hours,activity_level,activity_level_label,season
0,2012-09-23 20:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-09-23 21:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-09-23 22:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-09-23 23:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-09-24 00:00:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
df_energy = pd.read_csv('df_energy.csv')
df_energy.head()

,datetime,pittsburgh_temp,chicago_temp,Duquesne_kW,ComEdison_kW,is_NWD
0,2012-10-01 08:00:00-04:00,NaN,NaN,2.716667,2.82550,0
1,2012-10-01 09:00:00-04:00,7.850000,10.860000,2.733333,2.85650,0
2,2012-10-01 10:00:00-04:00,7.874767,10.904691,2.745000,2.88050,0
3,2012-10-01 11:00:00-04:00,7.938319,11.027412,2.716667,2.89350,0
4,2012-10-01 12:00:00-04:00,8.001870,11.150133,2.711667,2.87225,0


In [221]:
# convert date to datetime
df_energy['datetime'] = pd.to_datetime(df_energy['datetime'], utc=True)

# # remove timezone
# df_energy['datetime'] = df_energy['datetime'].dt.tz_localize(None)

# check
df_energy['datetime'].describe()

count                                  45246
mean     2015-05-02 06:54:36.727224320+00:00
min                2012-10-01 12:00:00+00:00
25%                2014-01-15 22:15:00+00:00
50%                2015-05-02 07:30:00+00:00
75%                2016-08-15 15:45:00+00:00
max                2017-11-30 00:00:00+00:00
Name: datetime, dtype: object

In [222]:
# get initial and final date
start_date = df_energy['datetime'].min()
end_date = df_energy['datetime'].max()

# remove lower than start date and higher than end date
df_pennsylvania_flu_hour = df_pennsylvania_flu_hour[(df_pennsylvania_flu_hour['date'] >= start_date) & (df_pennsylvania_flu_hour['date'] <= end_date)]
df_illinois_flu_hour = df_illinois_flu_hour[(df_illinois_flu_hour['date'] >= start_date) & (df_illinois_flu_hour['date'] <= end_date)]

# check
print(df_pennsylvania_flu_hour['date'].describe())

# export the datasets removing the index
df_pennsylvania_flu_hour.to_csv('df_pennsylvania_flu_hour.csv', index=False)
df_illinois_flu_hour.to_csv('df_illinois_flu_hour.csv', index=False)
df_energy.to_csv('df_energy_hour_no_tz.csv', index=False)

count                                  45254
mean     2015-05-02 02:19:06.966013952-04:00
min                2012-10-01 08:00:00-04:00
25%                2014-01-15 16:15:00-05:00
50%                2015-05-02 02:30:00-04:00
75%                2016-08-15 11:45:00-04:00
max                2017-11-29 19:00:00-05:00
Name: date, dtype: object
